### Import Required Package

In [1]:
import pandas as pd
import talib
import numpy as np
import json

### Import Tidal

In [2]:
import tidal as td

/opt/conda/envs/py310/lib/python3.10/site-packages/pyfolio/pos.py:25: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


### Initialize Plumber

In [3]:
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

PLUMBER_HOST = "https://dev-api.ddt-dst.cc/api/plumber/"
with open(f'{str(Path.home())}/.config/gcloud/application_default_credentials.json') as plumber_token:
    token = json.load(plumber_token)

### Load market data and compute technical indicators

In [4]:
import os
import pandas as pd

os.chdir('/home/jovyan/individualized-indicator')
# 定義你想要讀取檔案的目錄
directory = './emb'

start_date = "2022-06-01"
end_date = "2024-09-30"

# 初始化一個空的字典來存儲所有讀取的 DataFrame
dataframes = {}

# 遍歷目錄中的所有檔案
for filename in os.listdir(directory):
    if filename.endswith('.pkl'):
        # 組合完整的檔案路徑
        file_path = os.path.join(directory, filename)
        # 讀取 .pkl 檔案
        df = pd.read_pickle(file_path)
        
        # 使用檔案名稱（不包括副檔名）作為字典的鍵
        variable_name = filename.split('_')[2]
        
        # 將 DataFrame 儲存到字典中
        dataframes[variable_name] = df

keys_list = list(dataframes.keys())
# 將字符串轉換為 datetime 格式，然後進行排序
sorted_dates = sorted(pd.to_datetime(keys_list))
keys_list = [date.strftime('%Y-%m-%d') for date in sorted_dates]

total_df = pd.DataFrame()

for j in range(0,len(keys_list)):
    df_name = keys_list[j]
    selected_df = dataframes[df_name]    
    selected_df = selected_df.reset_index()
    stock_list = pd.DataFrame(selected_df.iloc[:,0].unique())
    total_df = pd.concat([total_df, stock_list], ignore_index=True)
    
unique_values_list = list(total_df.iloc[:,0].unique())
sorted_list = sorted(unique_values_list)

total_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": ",".join([stock for stock in sorted_list]),
    },
)

total_data.index = total_data.index.set_levels(
    pd.to_datetime(total_data.index.levels[1]),level=1)

total_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

total_data['new'] = -1
total_data.reset_index(inplace=True)

def filter_by_year_month_12(filter_data, select_year, select_mon):
    # 計算當前篩選年份和下一個年份
    next_year = select_year + 1
    
    # 構建篩選條件：當前年12月、次年1月、次年2月
    filter_data = filter_data[
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 1)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 2)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 3))
    ]
    
    return filter_data

concat_df = pd.DataFrame()

# for i in range(0,len(keys_list)-1):
for i in range(0,len(keys_list)):
    df_name = keys_list[i]
    selected_df = dataframes[df_name]
    selected_df.reset_index(inplace=True)
    stock_df = selected_df.iloc[:,0]
    stock_df_unique = list(stock_df.unique())
    result = list(set(sorted_list) - set(stock_df_unique))
    filtered_df = total_data[total_data['instrument'].isin(result)]
    filtered_df = filtered_df.rename(columns={'volume': 'vol'})
    selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
    
    select_year = int(df_name.split('-')[0])
    select_mon = int(df_name.split('-')[1])
    select_mons = [(select_mon+1)%12, (select_mon+2)%12, (select_mon+3)%12]
    select_mons = [12 if mon == 0 else mon for mon in select_mons]
    
    selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
    # 過濾出所有6月的交易資料
    filter_data = selected_df[selected_df['datetime'].dt.month.isin(select_mons)]

    # 使用時，根據 select_year 和 select_mon 動態篩選數據
        
    if select_mon == 12:
        filter_data = filter_by_year_month_12(filter_data, select_year, select_mon) 

    else:
        filter_data = filter_data[filter_data['datetime'].dt.year == select_year]
    
    concat_df = pd.concat([concat_df, filter_data], ignore_index=True)
    
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

concat_df.sort_values(by=['instrument', 'datetime'], inplace=True)
concat_df.reset_index(drop=True, inplace=True)
row = np.where(concat_df.iloc[:,0] == '6251')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '8406')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '6548')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
quote_data = concat_df.set_index(['instrument', 'datetime'])
quote_data.columns.values[4] = 'volume'

# quote_data.to_csv('./quote_data.csv')

/tmp/ipykernel_18577/4042907721.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
/tmp/ipykernel_18577/4042907721.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
/tmp/ipykernel_18577/4042907721.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_d

In [5]:
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

### Strategy Lake API

In [6]:
data_api_url = ("https://dev-api.ddt-dst.cc/api/plumber/nextron/industry_prod_df")
df = pd.read_parquet(
    data_api_url,
    storage_options={
        "gcp-token": json.dumps(token),
    }
)

### Load benchmark data

In [7]:
benchmark_inst = "0050"
benchmark_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": benchmark_inst,
    },
)
benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),
    level=1,
    inplace=True,
)
benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data

/tmp/ipykernel_18577/2173060872.py:11: FutureWarning: inplace is deprecated and will be removed in a future version.
  benchmark_data.index.set_levels(


open      high       low     close    volume
instrument datetime                                                    
0050       2022-06-01  119.2856  119.9775  119.0089  119.1473   6730117
           2022-06-02  118.5015  118.5015  117.7173  117.9018   4721923
           2022-06-06  118.3170  119.0089  117.5789  118.7321   4292361
           2022-06-07  118.0402  118.0402  117.0254  117.5328   5160242
           2022-06-08  118.5476  119.1011  118.2247  118.8244   4110313
...                         ...       ...       ...       ...       ...
           2024-09-24  183.2000  185.0000  182.1000  185.0000  10476465
           2024-09-25  187.7000  188.0000  187.3500  187.7500  12464313
           2024-09-26  189.6000  190.0000  188.7500  189.3000  17369543
           2024-09-27  190.2000  191.2500  188.6000  188.9000  11999510
           2024-09-30  186.5500  187.2000  183.9500  183.9500  14461065

[569 rows x 5 columns]

### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [8]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=50000000, slip_ticks=1, stock_config=td.StockConfig.TW, load_configs=True, reqMem="1000Mi", ignore_volume_size=True)

# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(10, 1, 'new'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data.loc[benchmark_inst]))

### Stock Config

In [9]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [10]:
tidal.exchange.inst_configs

{'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0}}

### Config Modification

In [11]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [12]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [13]:
tidal.backtest()
logs = tidal.get_logs()
for log in logs:
    print(log)

Tidal Backtesting: 100%|██████████| 548/548 [01:26<00:00,  6.31it/s, cash=4.21e+5, pnl=5.22e+5, position_cost=7.4e+7, value=7.5e+7]   

2023-10-17 - 警告：強制以開盤價 26.8733 賣出 5220，因為當天交易量為0
2024-08-27 - 訂單未能執行: 6591, 可能是由於現金不足或其他原因


### Metric - AccountInfo

In [14]:
account_info = tidal.metrics["AccountInfo"].report
print(account_info)

                    cash    trade_cost  position_cost     slip_cost  \
datetime                                                              
2022-07-01  5.000000e+07  0.000000e+00   0.000000e+00  0.000000e+00   
2022-07-04  2.970415e+06  1.540890e+04   4.701418e+07 -1.921235e+05   
2022-07-05  3.253694e+05  3.359008e+04   4.967433e+07 -1.311099e+05   
2022-07-06  3.606799e+05  6.031053e+04   4.956252e+07 -1.340187e+05   
2022-07-07  2.746189e+05  7.765470e+04   4.961224e+07 -1.661709e+05   
...                  ...           ...            ...           ...   
2024-09-24  3.042168e+05  1.145362e+07   7.384884e+07  8.858695e+06   
2024-09-25  3.042168e+05  1.145362e+07   7.384884e+07  8.858695e+06   
2024-09-26  2.180528e+05  1.147064e+07   7.383922e+07  8.896636e+06   
2024-09-27  5.864867e+05  1.150161e+07   7.398653e+07  8.780236e+06   
2024-09-30  4.212552e+05  1.153098e+07   7.403731e+07  8.840197e+06   

                     pnl         value  max_drawdown      turnover  
datetim

### Metric - PositionInfo

In [15]:
position_df = tidal.metrics["PositionInfo"].report
print(position_df)

                       quantity       price   commission    slip_cost  \
instrument datetime                                                     
1319       2024-04-30   52000.0  119.221667  2031.894877   37526.7048   
           2024-05-21   59000.0  125.755100  2431.757807  199756.3000   
           2024-06-05   64000.0  114.375194  2399.134065   76518.0032   
           2024-06-12   88000.0  116.313763  3354.721558  105552.7616   
           2024-06-13   88000.0  116.313763  3354.721558  105552.7616   
...                         ...         ...          ...          ...   
9942       2024-04-10   36000.0  127.972200  1509.943988       0.0000   
           2024-04-11   36000.0  127.972200  1509.943988       0.0000   
           2024-04-12   36000.0  127.972200  1509.943988       0.0000   
           2024-04-15   36000.0  127.972200  1509.943988       0.0000   
           2024-04-16   36000.0  127.972200  1509.943988       0.0000   

                               pnl  now_price  
in

In [16]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

,,quantity,price,commission,slip_cost,pnl,now_price
instrument,datetime,,,,,,


### Metric - AdditionalInfo

In [17]:
tidal.metrics['AdditionalInfo'].report

new
instrument datetime            
1101       2022-07-01  0.000566
           2022-07-04  0.000193
           2022-07-05  0.000557
           2022-07-06  0.000054
           2022-07-07  0.000682
...                         ...
9958       2024-09-24  0.001409
           2024-09-25  0.000066
           2024-09-26  0.000557
           2024-09-27  0.000468
           2024-09-30  0.000280

[332693 rows x 1 columns]

### Strategy Lake Submit

In [18]:
# group = dict(zip(df["coid"].astype(str), df["tejind4_c"]))
# submit_lake_backtest_result = tidal.submit_lake_backtest(account_info=account_info, position_df=position_df, benchmark_info=benchmark_data, group=group,lake_env="dev",strategy_id=19)
# print(submit_lake_backtest_result)

### Trade Report

In [19]:
tidal.trade_report

,win_num,lose_num,trade_num,pos_num,win_rate,profit,loss,trade_cost,pnl
instrument,,,,,,,,,
2392,2.0,0.0,2.0,0.0,1.000000,4.087092e+06,0.000000e+00,5.952402e+04,4.087092e+06
3617,6.0,6.0,12.0,0.0,0.500000,5.024968e+06,-1.023250e+06,2.712286e+05,4.001718e+06
3338,1.0,0.0,1.0,0.0,1.000000,3.493145e+06,0.000000e+00,4.539296e+04,3.493145e+06
1522,1.0,0.0,1.0,0.0,1.000000,3.125778e+06,0.000000e+00,3.251983e+04,3.125778e+06
3622,2.0,3.0,5.0,0.0,0.400000,3.374512e+06,-9.806210e+05,1.462943e+05,2.393891e+06
...,...,...,...,...,...,...,...,...,...
6788,4.0,8.0,12.0,0.0,0.333333,6.858106e+05,-1.808143e+06,2.586602e+05,-1.122333e+06
6667,0.0,7.0,7.0,0.0,0.000000,0.000000e+00,-1.167516e+06,1.290260e+05,-1.167516e+06
5236,1.0,2.0,3.0,0.0,0.333333,1.896723e+05,-1.885012e+06,6.341502e+04,-1.695340e+06


### Traded instruments

In [20]:
tidal.account.trades.keys()

dict_keys(['3669', '2640', '3617', '8433', '6245', '1817', '4549', '5230', '1531', '1580', '8928', '4129', '4971', '3551', '6829', '6568', '8027', '1533', '8109', '6667', '3556', '3567', '1565', '8091', '5220', '5439', '4945', '3141', '4506', '6591', '8279', '3611', '4541', '5903', '4175', '4938', '5284', '8942', '2404', '6664', '6788', '6470', '3413', '4927', '1589', '6670', '4961', '3264', '6143', '5443', '8049', '8420', '6570', '8210', '3038', '3029', '6139', '4557', '6691', '5511', '4137', '2916', '5508', '8048', '3356', '6782', '4138', '6776', '6538', '8112', '5236', '9942', '6189', '6585', '6284', '6523', '1522', '3213', '6170', '3227', '8081', '2392', '2636', '9927', '3338', '6605', '2371', '3622', '3712', '8054', '1532', '6546', '3211', '4915', '1319', '5309', '6290', '8255', '3689', '3357', '3624', '3045', '1722', '4105', '5340', '6579', '3289', '6741', '3147'])

### Trading History

In [21]:
# tidal.account.trades['3015']

### Plot chart by using Plotly

In [22]:
# tidal.analyzer.inst_chart(instrument='3015', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

### Tidal Dashboard

In [23]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:36967
 * Running on http://10.136.1.134:36967
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:10.0.10.82 - - [25/Nov/2024 15:22:16] "GET / HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [25/Nov/2024 15:22:16] "GET /static/js/main.d754b0a3.js HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [25/Nov/2024 15:22:18] "GET /static/css/main.bf4d504b.css HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [25/Nov/2024 15:22:19] "GET /images/Tidal_Logo_white.png HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [25/Nov/2024 15:22:19] "GET /api/quote/inst_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [25/Nov/2024 15:22:19] "GET /api/metric/metric_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [25/Nov/2024 15:22:19] "GET /api/trade/trade_report HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [25/Nov/2024 15:22:

In [ ]:
tidal.account.position_history

In [ ]:
Portfolio_info = tidal.metrics["Portfolio"].report
print(Portfolio_info)

In [ ]:
Portfolio_info.to_csv('./Portfolio_info.csv')